In [ ]:
%pip install keras

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch                                                              # PyTorchd
import pandas as pd                                                       # Pandas for data manipulation
import seaborn as sns                                                     # Seaborn for data visualization
import os                                                                 # OS library for file handling
import tensorflow as tf                                                   # TensorFlow for deep learning
import random                                                             # Random library for random number generation  
import keras
from tensorflow.keras.preprocessing.text import Tokenizer                            # This is going to help us tokenize the text
from keras.preprocessing.sequence import pad_sequences                    # This is going to help us pad the sequences
from keras.models import Sequential                                       # This is going to help us build the model
from textblob import TextBlob                                             # This is going to help us fix spelling mistakes in texts
from sklearn.feature_extraction.text import TfidfVectorizer               # This is going to helps identify the most common terms in the corpus
import re                                                                 # This library allows us to clean text data
import nltk                                                               # Natural Language Toolkit
nltk.download('punkt')                                                    # This divides a text into a list of sentences

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\UAE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
# Configuring Pandas to exhibit larger columns
'''
This is going to allow us to fully read the dialogues and their summary 
'''
pd.set_option('display.max_colwidth', 1000)

In [45]:
# Loading data
train = pd.read_csv(r"C:\Users\UAE\Downloads\Abdur Rafay\Datasets\Summarization Tool Dataset\samsum-train.csv")
test = pd.read_csv(r"C:\Users\UAE\Downloads\Abdur Rafay\Datasets\Summarization Tool Dataset\samsum-test.csv")
val = pd.read_csv(r"C:\Users\UAE\Downloads\Abdur Rafay\Datasets\Summarization Tool Dataset\samsum-validation.csv")

In [46]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14732 entries, 0 to 14731
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        14732 non-null  object
 1   dialogue  14731 non-null  object
 2   summary   14732 non-null  object
dtypes: object(3)
memory usage: 345.4+ KB


In [47]:
train = train.dropna() # removing null values

In [48]:
val = val.dropna() # removing null values

In [49]:
print(test['dialogue'].iloc[0])

Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye


In [51]:
def clean_tags(text):
    clean = re.compile('<.*?>') # Compiling tags
    clean = re.sub(clean, '', text) # Replacing tags text by an empty string

    # Removing empty dialogues
    clean = '\n'.join([line for line in clean.split('\n') if not re.match('.*:\s*$', line)])

    return clean

<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
C:\Users\UAE\AppData\Local\Temp\ipykernel_25452\185632439.py:6: SyntaxWarning: invalid escape sequence '\s'
  clean = '\n'.join([line for line in clean.split('\n') if not re.match('.*:\s*$', line)])


In [52]:
test1 = clean_tags(train['dialogue'].iloc[14727]) # Applying function to example text
test2 = clean_tags(test['dialogue'].iloc[0]) # Applying function to example text

# Printing results
print(test1)
print('\n' *3)
print(test2)

Theresa: Hey Louise, how are u?
Theresa: This is my workplace, they always give us so much food here 😊
Theresa: Luckily they also offer us yoga classes, so all the food isn't much of a problem 😂
Louise: Hey!! 🙂 
Louise: Wow, that's awesome, seems great 😎 Haha
Louise: I'm good! Are you coming to visit Stockholm this summer? 🙂
Theresa: I don't think so :/ I need to prepare for Uni.. I will probably attend a few lessons this winter
Louise: Nice! Do you already know which classes you will attend?
Theresa: Yes, it will be psychology :) I want to complete a few modules that I missed :)
Louise: Very good! Is it at the Uni in Prague?
Theresa: No, it will be in my home town :)
Louise: I have so much work right now, but I will continue to work until the end of summer, then I'm also back to Uni, on the 26th September!
Theresa: You must send me some pictures, so I can see where you live :) 
Louise: I will, and of my cat and dog too 🤗
Theresa: Yeeeesss pls :)))
Louise: 👌👌
Theresa: 🐱💕




Hannah: He

In [53]:
# Defining function to clean every text in the dataset.
def clean_df(df, cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(clean_tags)
    return df

In [54]:
# Cleaning texts in all datasets
train = clean_df(train,['dialogue', 'summary'])
test = clean_df(test,['dialogue', 'summary'])
val = clean_df(val,['dialogue', 'summary'])

In [55]:
train.tail(3) # Visualizing results

,id,dialogue,summary
14729,13819050,"John: Every day some bad news. Japan will hunt whales again\r\nErica: Yes, I've read this. It's very upsetting\r\nJohn: Cruel Japanese\r\nFaith: I think this is a racist remark. Because Island and Norways has never joined this international whaling agreement\r\nErica: really? I haven't known, everybody is so outraged by Japan\r\nFaith: sure, European hypocrisy \r\nJohn: not entirely. Scandinavians don't use the nets that Japanese use, so Norway and Island kill much less specimens that Japan will\r\nFaith: oh, it's much more complex than one may expect\r\nJohn: True, but the truth is, that all of them should stop\r\nJohn: and this decision is a step back\r\nFaith: yes, this is worrying\r\nErica: And it seems that the most important whaling countries are out of the agreement right now\r\nFaith: yes, seems so\r\nJohn: Just like USA leaving the Paris Agreement",Japan is going to hunt whales again. Island and Norway never stopped hunting them. The Scandinavians kill fewer whales than the Japanese.
14730,13828395,"Jennifer: Dear Celia! How are you doing?\r\nJennifer: The afternoon with the Collins was very pleasant, nice folks, but we missed you.\r\nJennifer: But I appreciate your consideration for Peter.\r\nCelia: My dear Jenny! It turns out that my decision not to come, though I wanted so much to see you again and Peter and the Collins, was right. Yesterday it all developed into a full bore cold. Sh.....\r\nCelia: All symptoms like in a text book.\r\nCelia: Luckily it's contagious only on the first 2, 3 days, so when we meet next week it should be alright.\r\nCelia: Thanks for asking! Somehow for all of us Peter comes first now.\r\nJennifer: That's too bad. Poor you...\r\nJennifer: I'll be driving to FR, do you want me to bring you sth? It's on my way.\r\nCelia: Thank you dear! I was at the pharmacy yesterday and had done my shopping the day before.\r\nCelia: You'd better still stay away from me in case I'm still contagious\r\nJennifer: Right. So I'll only leave a basket on your terrace. A...","Celia couldn't make it to the afternoon with the Collins and Jennifer as she is ill. She's working, but doesn't want to meet with Jennifer as it might be contagious. Jennifer will leave a basket with cookies on Celia's terrace."
14731,13729017,"Georgia: are you ready for hotel hunting? We need to book something finally for Lisbon\r\nJuliette: sure we can go on, show me what you found\r\nJuliette: nah... it looks like an old lady's room lol\r\nJuliette: that's better... but the bed doesn't look very comfortable\r\nGeorgia: i kind of like it and it's really close to the city center\r\nJuliette: show me the others please\r\nJuliette: nah... this one sucks too, look at those horrible curtains \r\nGeorgia: aff Julie you are such a princess\r\nJuliette: i just want to be comfortable\r\nGeorgia: come on, stop whining you know we are on a budget\r\nJuliette: well hopefully we can find something that's decent right?\r\nGeorgia: i did show you decent but you want a Marriott or something :/\r\nJuliette: ok ok don't get angry\r\nGeorgia: we need to decide today, the longer we wait the higher the prices get \r\nJuliette: ok how about we get the second one then?\r\nGeorgia: ok give me a second\r\nJuliette: sure\r\nGeorgia: affff someon...","Georgia and Juliette are looking for a hotel in Lisbon. Juliette dislikes Georgia's choices. Juliette and Georgia decide on the second option presented by Georgia, but it has already been booked. Finally Georgia books the third hotel."


In [1]:
# Extract dialogues and summaries
train_dialogues = train['dialogue'].tolist()
train_summaries = train['summary'].tolist()
val_dialogues = val['dialogue'].tolist()
val_summaries = val['summary'].tolist()

# Fit tokenizer on both dialogues and summaries
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(train_dialogues + train_summaries + val_dialogues + val_summaries)

# Convert texts to sequences
train_dialogue_seq = tokenizer.texts_to_sequences(train_dialogues)
train_summary_seq = tokenizer.texts_to_sequences(train_summaries)
val_dialogue_seq = tokenizer.texts_to_sequences(val_dialogues)
val_summary_seq = tokenizer.texts_to_sequences(val_summaries)

# Set max sequence lengths
MAX_DIALOGUE_LEN = 256
MAX_SUMMARY_LEN = 32

# Pad or truncate sequences
train_dialogue_seq = pad_sequences(train_dialogue_seq, maxlen=MAX_DIALOGUE_LEN, padding='post', truncating='post')
train_summary_seq = pad_sequences(train_summary_seq, maxlen=MAX_SUMMARY_LEN, padding='post', truncating='post')
val_dialogue_seq = pad_sequences(val_dialogue_seq, maxlen=MAX_DIALOGUE_LEN, padding='post', truncating='post')
val_summary_seq = pad_sequences(val_summary_seq, maxlen=MAX_SUMMARY_LEN, padding='post', truncating='post')

NameError: name 'train' is not defined

In [ ]:
from torch.utils.data import Dataset, DataLoader

class SummarizationDataset(Dataset):
    def __init__(self, dialogues, summaries):
        self.dialogues = dialogues
        self.summaries = summaries

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        dialogue = torch.tensor(self.dialogues[idx], dtype=torch.long)
        summary = torch.tensor(self.summaries[idx], dtype=torch.long)
        return dialogue, summary

# Create datasets
train_dataset = SummarizationDataset(train_dialogue_seq, train_summary_seq)
val_dataset = SummarizationDataset(val_dialogue_seq, val_summary_seq)

# Create dataloaders
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Instead of passing DataFrames, use the DataLoader to get batches of tokenized and padded tensors
for batch_idx, (dialogue_batch, summary_batch) in enumerate(train_loader):
    # dialogue_batch and summary_batch are already tensors of sequences
    print(f"Batch {batch_idx}:")
    print("Dialogue batch shape:", dialogue_batch.shape)
    print("Summary batch shape:", summary_batch.shape)
    break  # Remove this break to iterate through all batches

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.word_index['<PAD>'])

num_epochs = 20  # You can adjust as needed

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for dialogue_batch, summary_batch in train_loader:
        optimizer.zero_grad()
        # Forward pass: input is dialogue, target is summary
        # For teacher forcing, pass summary_batch[:, :-1] as decoder input, predict summary_batch[:, 1:]
        outputs = model(dialogue_batch, summary_batch[:, :-1])  # You need to update your model to accept decoder input
        # outputs: (batch, seq_len, vocab_size)
        loss = criterion(outputs.reshape(-1, outputs.size(-1)), summary_batch[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')


NameError: name 'tokenizer' is not defined

In [ ]:
# Function to generate summaries from new dialogue sequences
def generate_summary(model, dialogue_seq, tokenizer, max_summary_len=32, device='cpu'):
    model.eval()
    input_seq = torch.tensor(dialogue_seq).unsqueeze(0).to(device)  # Add batch dimension
    summary_seq = [tokenizer.word_index.get('<START>', 1)]  # Start token
    for _ in range(max_summary_len):
        summary_tensor = torch.tensor(summary_seq).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_seq, summary_tensor)
            next_token = output.argmax(-1)[:, -1].item()
        if next_token == tokenizer.word_index.get('<END>', 2) or next_token == 0:
            break
        summary_seq.append(next_token)
    # Convert token ids back to words
    inv_vocab = {v: k for k, v in tokenizer.word_index.items()}
    summary_words = [inv_vocab.get(idx, '') for idx in summary_seq[1:]]
    return ' '.join(summary_words)

# Example usage:
# new_dialogue = "Alice: Hi Bob!\nBob: Hello Alice, how are you?"
# new_seq = tokenizer.texts_to_sequences([new_dialogue])
# new_seq = pad_sequences(new_seq, maxlen=256, padding='post', truncating='post')
# print(generate_summary(model, new_seq[0], tokenizer))

# Saving model checkpoint
def save_checkpoint(model, optimizer, epoch, path='model_checkpoint.pth'):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch
    }, path)

# Loading model checkpoint
def load_checkpoint(model, optimizer, path='model_checkpoint.pth'):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return checkpoint['epoch']

In [40]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out, hn, cn